### Packages

In [1]:
import sys
import torch
import pandas as pd
sys.path.append('../..')
from torch.utils.data import DataLoader
from src.training.train import load_config,create_transfroms,create_model,Collate
from src.datasets import ISICDataset
from src.utils import *
from definitions import *
from tqdm.notebook import tqdm

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.13 (you have 1.4.8). Upgrade using: pip install --upgrade albumentations


### Load configurations

In [3]:
EXPIREMENT = 'efficientnet_b0'

In [4]:
configs = load_config(EXPIREMENT)

### Load the model

In [5]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
def load_models(expirement_name : str) -> list:
    

    expirement_dir = os.path.join(EXPIREMENTS_DIR,expirement_name)
    checkpoints_dir = os.path.join(expirement_dir,'checkpoints')

    models = []

    files = os.listdir(checkpoints_dir)
    files = sorted(files,key = lambda x: int(x.split('_')[-1]))

    for fold in files:

        print(f'Loading model from {fold}')

        fold_dir = os.path.join(checkpoints_dir,fold)
        model_name = os.listdir(fold_dir)[0]

        model_path = os.path.join(fold_dir,model_name)

        model = create_model(configs)
        model.load_state_dict(torch.load(model_path))
        model.to(DEVICE)
        model.eval()

        models.append(model)
    
    return models

In [7]:
models = load_models(EXPIREMENT)

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b0.ra_in1k)


Loading model from fold_0


INFO:timm.models._hub:[timm/efficientnet_b0.ra_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b0.ra_in1k)


Loading model from fold_1


INFO:timm.models._hub:[timm/efficientnet_b0.ra_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b0.ra_in1k)


...

INFO:timm.models._hub:[timm/efficientnet_b0.ra_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b0.ra_in1k)


Loading model from fold_3


INFO:timm.models._hub:[timm/efficientnet_b0.ra_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b0.ra_in1k)


Loading model from fold_4


INFO:timm.models._hub:[timm/efficientnet_b0.ra_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


### Create dataloaders

In [8]:
train_transfroms,val_transfroms = create_transfroms(configs)

In [9]:
dataset = ISICDataset(
    hdf5_file=os.path.join(ISIS_2024_DIR, 'images.hdf5'),
    metadata_file=os.path.join(ISIS_2024_DIR, 'metadata.csv'),
    img_transform=val_transfroms,
    return_metadata=False
)

In [10]:
loader = DataLoader(
    dataset=dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4,
    prefetch_factor=2,
    collate_fn=Collate(configs, False)
)

### Predicting with TTA

In [11]:
transfroms = {
    'rotate_0' : lambda x : x,
    # 'rotate_90' : lambda x : torch.rot90(x,1,[2,3]),
    # 'rotate_180' : lambda x : torch.rot90(x,2,[2,3]),
    # 'rotate_270' : lambda x : torch.rot90(x,3,[2,3]),
    # 'horizontal_flip' : lambda x : torch.flip(x,[3]),
    # 'vertical_flip' : lambda x : torch.flip(x,[2]),
    # 'horizontal_vertical_flip' : lambda x : torch.flip(x,[2,3]),
    # 'transpose' : lambda x : torch.transpose(x,2,3)
}

In [12]:
def feature_extract(models : list[torch.nn.Module],loader : DataLoader,transfroms : list) -> pd.DataFrame:
    
    df = dict()
    df['isic_id'] = loader.dataset.metadata['isic_id'].values

    for fold,_ in enumerate(models):
        for transfrom in transfroms:
            df[f'fold_{fold}_{transfrom}'] = []

    with torch.no_grad():

        for x,y in tqdm(loader):

            x = x.to(DEVICE)

            for transfrom_name,transfrom in transfroms.items():

                transfromed_x = transfrom(x)

                for fold,model in enumerate(models):
                    
                    predictions = model(transfromed_x)

                    df[f'fold_{fold}_{transfrom_name}'].extend(predictions.detach().cpu().numpy())

    return pd.DataFrame(df)

In [13]:
df = feature_extract(models,loader,transfroms)

  0%|          | 0/12534 [00:00<?, ?it/s]

In [14]:
df.to_csv(os.path.join(EXPIREMENTS_DIR,EXPIREMENT,'features.csv'),index=False)